# Extract One line of Data from the XML File

In [14]:
import os
os.system("sed -n '2p' PRD.REV.TU.DWH.REGULAR_230412_090000_0000001.xml > ./output/entity.xml")

0

# Convert `entity.xml` to `entity.json` 

In [15]:
import sys
import xmltodict
import json
import xml

def convert_xml_to_json(xml_file):
    try:
        with open(xml_file, 'r') as f:
            xml_data = f.read()

        json_data = xmltodict.parse(xml_data, force_list={'item'}) 
        return json_data
    except FileNotFoundError:
        print(f"Error: File '{xml_file}' not found.")
        sys.exit(1)
    except xml.parsers.expat.ExpatError as e:
        print(f"Error parsing XML: {e}")
        sys.exit(1)

xml_file = "./output/entity.xml"
json_data = convert_xml_to_json(xml_file)

with open('./output/entity.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)
print("Conversion complete. JSON data written to output.json.")


Conversion complete. JSON data written to output.json.


# Determine the equivelent Relational Schema for the json File

In [16]:
import json

#define data detection
def infer_data_type(value):
    if isinstance(value, int):
        return "NUMBER"
    elif isinstance(value, float):
        return "NUMBER"
    elif isinstance(value, bool):
        return "BOOLEAN"
    elif isinstance(value, str):
        return "VARCHAR2(255 BYTE)"
    elif isinstance(value, dict):
        return "INT"
    elif isinstance(value, list):
        return "INT"
    else:
        return "VARCHAR2(255 BYTE)" 

def json_to_sql(json_data):
    tables = {}
    dependencies = {}

    def process_data(data, table_name=''):
        nonlocal tables, dependencies
        if isinstance(data, dict):
            if not table_name:
                table_name = 'main'
                tables[table_name] = {}
            for key, value in data.items():
                column_name = key.replace('@', '')
                column_type = infer_data_type(value)
                if isinstance(value, dict) or isinstance(value, list):
                    sub_table_name = key[:30].replace('@', '')
                    if sub_table_name not in tables:
                        tables[sub_table_name] = {}
                    tables[table_name][column_name] = f"{column_type} REFERENCES {sub_table_name}(id)"
                    dependencies[sub_table_name] = dependencies.get(sub_table_name, 0) + 1
                    process_data(value, sub_table_name)
                else:
                    tables[table_name][column_name] = column_type
        elif isinstance(data, list):
            if len(data) > 0 and (isinstance(data[0], dict) or isinstance(data[0], list)):
                sub_table_name = f"{table_name}_list"
                if sub_table_name not in tables:
                    tables[sub_table_name] = {}
                tables[table_name][table_name[:30].replace('@', '')] = f"{infer_data_type(data[0])} REFERENCES {sub_table_name}(id)"
                dependencies[sub_table_name] = dependencies.get(sub_table_name, 0) + 1
                process_data(data[0], sub_table_name)

    process_data(json_data)

    # Sort tables based on their dependencies
    sorted_tables = sorted(tables.keys(), key=lambda x: dependencies.get(x, 0))

    sql_statements = []
    for table_name in sorted_tables:
        create_table_sql = f"CREATE TABLE {table_name} (\n"
        for column, data_type in tables[table_name].items():
            if column == 'id':
                create_table_sql += f"    {column} NUMBER(10,0) PRIMARY KEY GENERATED ALWAYS AS IDENTITY,\n"
            else:
                create_table_sql += f"    {column} {data_type.upper()},\n"
        create_table_sql = create_table_sql.rstrip(',\n') + "\n);"
        sql_statements.append(create_table_sql)

    return sql_statements

def main():
    with open('./output/entity.json', 'r') as file:
        json_data = json.load(file)

    sql_statements = json_to_sql(json_data)

    with open('./output/schema.sql' , 'a') as file:
        for statement in sql_statements:
            file.write(f"{statement}\n")

if __name__ == "__main__":
    main()


# Reverse the order of creation

In [17]:

with open('./output/schema.sql', 'r') as file:
    sql_content = file.read()

sql_queries = sql_content.split(';')


sql_queries = [query.strip() for query in sql_queries if query.strip()]

sql_queries.reverse()

reversed_sql_content = ';\n'.join(sql_queries)

with open('./output/schema.sql', 'w') as file:
    file.write(reversed_sql_content)
